In [ ]:
"""
本文档说明：

也会删除特征，放在数据导入部分，离散化后删除原来特征

可视化决定如何进行离散化
然后借助等距来分箱（除非时间不够，不然等距等频用处不大）
信息熵等分箱更是像聚类一样杂，没有标准

离散编码一般就是独热
平均和WOE感觉更适合回归问题

日期处理pass

"""

#  数据导入

In [ ]:
%matplotlib inline
import numpy as np
import torch
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import missingno as msno
import dtale
from sklearn import preprocessing as pp
import sklearn

In [ ]:
fpath="C:/Users/Huai/Desktop/tool/train_v3_discrete_temp.csv"
df=pd.read_csv(fpath)

#简单数据查看、不必dtale
df.shape
df.head(5)
#df.info()

In [ ]:
msno.bar(df=df, figsize=(8, 4), fontsize=18) 

In [ ]:
dtale.show(df)

In [ ]:
#删除特征  #平均码可以减小一个维度
#df=df.drop(['PassengerId',"Name"],axis=1)
df=df.drop(['SibSp','Parch'],axis=1)

#  连续值离散化

##  可视化

In [ ]:
# 2.1连续值离散化（可视化）
import re
def hist_line(x,y,target_class,bins_num,color):
    #设置格式 
    sns.set_style("whitegrid")
    #设置图形大小
    plt.rcParams['figure.figsize'] = (18.0,8.0)
    fig = plt.figure()
    #画柱形图
    ax1 = fig.add_subplot(111)
    n,bins_rule,patches=ax1.hist(x, bins=bins_num, facecolor=color, edgecolor="black", alpha=0.5)
    ax1.set_ylabel(x.name, fontsize='15')
    ax1.set_title(x.name+"-y",fontsize='20')
    
    for i in range(len(bins_rule)):
        bins_rule[i]=round(bins_rule[i],3)
    plt.xticks(bins_rule,bins_rule, fontsize=15, rotation=45)#横坐标刻度，用rule设置

    #print(bins_rule) #[18.  25.7 33.4 41.1 48.8 56.5 64.2 71.9 79.6 87.3 95. ]
    height=[]
    target=[]
    target_name=y.name
    for i in range((len(bins_rule)-1)):
        height.append(   int(re.findall(r"height=(.+?),",str(patches[i]))[0])   )
        #print("rule:",bins_rule[i],bins_rule[i+1],"height:",height)  height=[["height1"],["height2"],["height3"]]
        new_ser=df.loc[ (x>=bins_rule[i]) & (x<bins_rule[i+1]), target_name]
        target.append(  new_ser[new_ser==target_class].count() )
        #target.append(df.loc[ (x>=bins_rule[i]) & (x<bins_rule[i+1]), target_name].sum())  #这个只能应用于目标为1/0  要修改！！！
        #print(target) #[169, 830, 730, 392, 362, 280, 81, 86, 27, 3]
    print("Height：",height)
    print("Target:",target)
    #画折线图 
    ax2 = ax1.twinx()   #组合图必须加这个
    line_x=[]
    line_y=[]
    for i in range((len(bins_rule)-1)):
        line_x.append(   round((bins_rule[i]+bins_rule[i+1])/2,2)      )
        line_y.append(   round((target[i]/(height[i]-target[i])),2)    ) 
    
    #print(line_x)
    print("y-1:0 :",line_y)
    ax2.plot(line_x, line_y, 'slateblue',ms=5, marker='o',linewidth=0.7)
    ax2.set_ylabel('y-1:0',fontsize='15')
    for a, b in zip(line_x, line_y):
        plt.text(a, b+0.0, b, ha='center', va='bottom', fontsize=15)
    ax2.grid(None)#不展示右轴的网格
    plt.show()



x = df["number"]
y = df["Survived"]
target_class=1#y中的阳性样本，最后的比值是（阳性样本数量/其他）
bins_num=[-1,0.5,3.5,50]#[18,25,30,60,95]#也可是一个数字
color="khaki"
hist_line(x,y,target_class,bins_num,color)
"输出 Height Target Y-1:0"
#记得保存规则！！！

##  分箱

###  等距分箱

In [ ]:
df["number"].describe()

In [ ]:
ser=df["number"].copy()
#class_bin对于等距 数字是分几个类别  list是把极值进行划分
#class_bin=5
class_bin=[-1,0.5,3.5,50]
class_label=["number_s","number_m","number_l"]

new_ser,rule=pd.cut(ser,bins=class_bin,right=True,labels=class_label,retbins=True)
print(new_ser)
print(new_ser.value_counts())

#记得保存规则！！！
print(rule)

In [ ]:
df["number"]=new_ser

###  等频分箱

In [ ]:
ser=df["Fare"].copy()
#class_bin对于等频 数字是分几个类别  list是把数量进行划分 前30% 50%类似 且范围[0-1]
class_bin=5
#class_bin=[0,0.25,0.5,0.75,0.8,1]
class_label=["one","two","three","four","five"]

new_ser,rule=pd.qcut(ser,q=class_bin,labels=class_label,retbins=True,duplicates="drop")
print(new_ser)
print(new_ser.value_counts())
print(rule)

##  信息熵分箱

###  信息熵分箱(详细代码方法：不方便使用)

In [ ]:
class entropy:
    @staticmethod
    def entropy(x):
        p=x.value_counts(normalize=True)
        p=p[p>0]
        e=-(p*np.log2(p)).sum()
        return e
    @staticmethod
    def cond_entropy(x,y):
        p=y.value_counts(normalize=True)
        e=0
        for yi in y.unique():
            e+=p[yi]*entropy.entropy(x[y==yi])
        return 0
    @staticmethod
    def info_gain(x,y):
        g=entropy.entropy(x)-entropy.cond_entropy(x,y)
        return g

def _is_too_small(df,min_count):
    return df.shape[0]<min_count
def _is_only_one_class(df,label):
    return len(df[label].unique().tolist())==1



def get_max_gain_point(df, label, feature_col):
    '''
    分箱后，使用离散方法计算信息增益
    注意这里的for实现效率较低，实践中请先粗分类
    '''
    gain, max_p = -1, -1
    ps = df[feature_col].unique().tolist()
    ps.sort()

    if len(ps) < 2:
        return -1, None

    for pp in ps:
        #tmp = (df[feature_col] <= pp).astype(int)
        tmp = (df[feature_col] < pp).astype(int)
        g = entropy.info_gain(df[label], tmp)
        if g > gain:
            gain = g
            max_p = pp
    return gain, max_p

def cut_by_entropy(df, label, loop=3, min_count=5, margin=0.01):
    '''停止准则：
    1、已到循环次数
    2、信息增益增加小于阈值（用最小熵，不便定义熵值的大小）
    '''
    assert len(df.columns) == 2, 'not support'

    def _get_best_points(df, label, feature_col, loop, min_count,  margin):
        if loop == 0 or _is_only_one_class(df,label) or _is_too_small(df,min_count):
            print("loop end")
            return [None]
        else:
            # 计算信息增益
            gain, max_p = get_max_gain_point(df, label, feature_col)
            print('max_p={},gain={}'.format(max_p, gain))
            # 信息增益小于指定阈值时停止分箱
            if gain < margin:
                return [None]

            # 左闭，右开
            left = df.loc[df[feature_col] < max_p, :]
            right = df.loc[df[feature_col] >= max_p, :]
            
            

            # 递归分箱
            return [max_p] + \
                _get_best_points(left, label, feature_col,loop - 1, min_count,  margin) + \
                _get_best_points(right, label, feature_col, loop - 1, min_count,  margin)

    feature_col = [aa for aa in df.columns.tolist() if aa != label][0]
    points = _get_best_points(df, label, feature_col, loop, min_count, margin)
    points = [p for p in points if p is not None]
    points = list(set(points))
    points.sort()
    return points

cut_by_entropy(df[["Fare","Survived"]],"Survived", loop=2, min_count=1, margin=0.1)

###  信息熵分箱(sklearn决策树方法)

In [ ]:
df["Fare"]

In [ ]:
from sklearn.tree import DecisionTreeClassifier

# max_depth=3，表示进行3次划分构造3层的树结构
def dt_entropy_cut(x, y, max_depth=3, criterion='entropy'):  
    dt = DecisionTreeClassifier(criterion=criterion, max_depth=max_depth)
    dt.fit(x.values.reshape(-1, 1), y)
    qts = dt.tree_.threshold[np.where(dt.tree_.children_left > -1)]
    if qts.shape[0] == 0:
        qts = np.array([np.median(data[:, feature])])
    else:
        qts = np.sort(qts)
    return qts

cutoff = dt_entropy_cut(df["Fare"], df["Survived"], max_depth=3, criterion='entropy')#gini  #------------可以修改参数
cutoff=cutoff.tolist()

#在cutoff加上两边才是真正的rule！！！
print(cutoff)

In [ ]:
#方便审查决策树分辨结果
ser=df["Fare"].copy()
class_bin=[-1000]+cutoff+[1000]#要给两边添加边界
class_label=["a","b","c","d","e","f","g","h"]

new_ser,rule=pd.cut(ser,bins=class_bin,right=True,labels=class_label,retbins=True)

#查看分箱后的类别数量
print(new_ser.value_counts())
"""
e    435
d    153
c    141
h     89
a     28
f     15
b     13
g      5
"""

##  卡方

In [ ]:
pass

##  Best-KS

In [ ]:
pass

#  离散值编码

## 定序；划分类（避免类别太多）

In [ ]:
df.info()

In [ ]:
print(df["SibSp"])
print("\n")
print(df["SibSp"].value_counts())

In [ ]:
#3.1定序值
ser=df["number"].copy()
#ser[ser=="Fare_s"]="a"#-------------0
#ser[ser=="Fare_m"]="b"#-------------1
#ser[ser=="Fare_l"]="c"#-------------2
name_list=["number_s","number_m","number_l"]
#按照字母序 a:0  b:1

trans=pp.LabelEncoder().fit(np.array(name_list))
new_ser=trans.transform(ser)
#return trans, new_ser

print(new_ser)

df["number"]=new_ser

## 独热编码

In [ ]:
print(df["Fare"])

In [ ]:
#3.2独热  #先定序后独热
ser=df["number"].copy()
temp = np.array(ser)
temp = temp.reshape(temp.shape[0], 1)
trans1=pp.OneHotEncoder().fit(temp)
trans2=trans1
temp = trans1.transform(temp).toarray()  #woc 后面加一个.toarray()就跑通了 https://blog.csdn.net/small__roc/article/details/122947001
new_df = pd.DataFrame(temp)
print(new_df)#这里列名0-1-2 和定序值中的0-1-2对应

In [ ]:
df[["Number_l","Number_m","Number_s"]]=new_df[[0,1,2]]
print(df)

In [ ]:
#删除特征
#df=df.drop(['PassengerId',"Name"],axis=1)
df=df.drop(['number'],axis=1)

## 平均编码

In [ ]:
print(df["Embarked"])

In [ ]:
#3.3平均码
#http://helios.mm.di.uoa.gr/~rouvas/ssi/sigkdd/sigkdd.vol3.1/barreca.pdf
feature="Embarked"
target="Survived"
k=400 #n=400 认为先验后验一样重要  #z=(n-k)/f
f=500   #坡度平缓 越大越缓慢

#获得先验
name_list=[feature,target]
mean_df = pd.DataFrame(columns=["feature", "target"])
mean_df[["feature","target"]]=df[name_list].copy()
#获得总行数/样本数
row_len=mean_df.shape[0]  
#获得先验概率
prior_df=mean_df["target"].value_counts(ascending=True).sort_index(ascending=True)/row_len#(ascending=True).sort_index(ascending=True)字母序
#0    0.620023
#1    0.379977
prior_index=(prior_df.index).tolist()
prior_list=prior_df.tolist()
#[0, 1]
#[0.6200227531285551, 0.3799772468714448]

#获得后验
feature_group = mean_df.groupby("feature")
#建立空字典
d={}
for key,value in feature_group:
    # key  是feature的类别
    # value就是相应的dataframe
    temp_row_len=value.shape[0]
    temp_posterior_df=value["target"].value_counts(ascending=True).sort_index(ascending=True)/temp_row_len
    #temp_posterior_index=(temp_posterior_df.index).tolist()  #不需要，因为和先验一致
    temp_posterior_list=temp_posterior_df.tolist()
    
    #获得λ
    n=temp_row_len
    z=(n-k)/f
    lambda_n=(  1/(1+np.exp(z))  )
    final_list=np.array([x*lambda_n for x in prior_list]) +  np.array([x*(1-lambda_n) for x in temp_posterior_list])
    
    print(value)
    print("\n")
    print("n:",temp_row_len)
    print("Z:",z)
    print("λ:",lambda_n)
    print("prior:",prior_list)
    print("posterior:",temp_posterior_list)
    print("final:",final_list)
    print("\n")
    print("----------------------------------")
    
    #装入字典
    d[key]=final_list
print(d)
    

In [ ]:
#获得平均码之后的特征名
mean_name_list=[]
for i in prior_index:
    mean_name_list=mean_name_list+["mean"+"_"+feature+"_"+str(i)]

#建立空df
new_df = pd.DataFrame(columns=mean_name_list)

#给new_df赋值
flag=0
for i in prior_index:
    new_df["mean"+"_"+feature+"_"+str(i)]=mean_df['feature'].apply(lambda x : d[x][flag])
    flag=flag+1

#展示和赋值
#print(new_df)
#df[mean_name_list]=new_df[mean_name_list]

In [ ]:
#删除特征  #平均码可以减小一个维度
#df=df.drop(['PassengerId',"Name"],axis=1)
df=df.drop(['Sex','Pclass',"Embarked"],axis=1)

### 平均编码(cross)

In [ ]:
#3.3平均码
#http://helios.mm.di.uoa.gr/~rouvas/ssi/sigkdd/sigkdd.vol3.1/barreca.pdf
feature="Embarked"
target="Survived"
k=400 #n=400 认为先验后验一样重要
f=500   #坡度平缓 越大越缓慢

#获得先验
name_list=[feature,target]
mean_df = pd.DataFrame(columns=["feature", "target"])
mean_df[["feature","target"]]=df[name_list].copy()

temp_cross=pd.crosstab(mean_df["feature"],mean_df["target"])
cross=temp_cross.div(temp_cross.sum(axis=1),axis=0)
print(temp_cross)
print(cross)

#也许会更好

## WOE编码

In [ ]:
print(df["Embarked"])

In [ ]:
#WOE
#现在对WOE码理解还不深，该处只适用于二分类
feature="Embarked"
target="Survived"

#获得先验
name_list=[feature,target]
woe_df = pd.DataFrame(columns=["feature", "target"])
woe_df[["feature","target"]]=df[name_list].copy()
#获得下分布
prior_df=woe_df["target"].value_counts(ascending=True).sort_index(ascending=True)#(ascending=True).sort_index(ascending=True)字母序
#0    545
#1    334
prior_num_list=prior_df.tolist()
#[545, 334]
prior_num_rate=prior_num_list[0]/prior_num_list[1]


#h获得后验
feature_group = mean_df.groupby("feature")
#建立空字典
d={}
for key,value in feature_group:
    # key  是feature的类别
    # value就是相应的dataframe
    temp_posterior_df=value["target"].value_counts(ascending=True).sort_index(ascending=True)
    temp_posterior_list=temp_posterior_df.tolist()
    temp_posterior_num_rate=temp_posterior_list[0]/temp_posterior_list[1]
    
    
    print(value)
    print("\n")
    print(temp_posterior_df)
    print(temp_posterior_list)
    print(temp_posterior_num_rate)
    print("\n")
    print("----------------------------------")
    
    #装入字典
    d[key]=np.log(temp_posterior_num_rate/prior_num_rate)
print(d)

In [ ]:
#获得WOE之后的特征名
#建立空df
new_df = pd.DataFrame(columns=["woe"+"_"+feature])

#给new_df赋值
new_df["woe"+"_"+feature]=woe_df['feature'].apply(lambda x : d[x])

#展示和赋值
print(new_df)
#df[mean_name_list]=new_df[mean_name_list]

# 日期处理

In [ ]:
#4.处理日期  #这里关键是可以对日期的字符串进行拆分
pass

# 数据导出

In [ ]:
fpath_out="C:/Users/Huai/Desktop/tool/train_v3_discrete.csv"
df.to_csv(fpath_out,encoding='utf-8',index=False)